## Part 1: Load Libaries and Import necessary dataset

In [ ]:
# https://otexts.com/fpp3/prophet.html
# Load required libraries. Make sure to install them too
library(readxl)
library(dplyr)
library(Rcpp)
library(ggplot2)
library(lubridate)
library(forecast)
library(prophet)
library(Metrics)
library(zoo) # use for date formatting 
library(stats) # use for additive
library(caret)
library(glmnet)


In [ ]:
# Extract deposit_levels to use it in this file
deposit_levels <- readRDS("/workspaces/DUKE_MIDS_QFC_Final_Project/data_cleaning/deposit_levels.rds") # Load the data


In [ ]:
# Extract regressors_data to use it in this file
regressors_data <- readRDS("/workspaces/DUKE_MIDS_QFC_Final_Project/data_cleaning/regressors_data.rds")

## Part 2: Data Exploration for Prophet Assumption

- **Purpose**: Prophet requires data to be regular (no gaps in time periods) and free of missing values to build accurate forecasts.

In [ ]:
sum(is.na(deposit_levels))  # no missing values

- The time series has regular quaterly intervals and no missing values so the data regurlarity expectation is satified. 

- It also has decades worth of data so the "sufficient historical data" expectation is satisfied. 

- **Purpose**: Prophet models require a clear understanding of the trend to accurately forecast future values. 

- **Next Steps**: Evaluate whether the data exhibits a linear or logistic trend.

- **Question** Does the choice of a linear trend or logistic trend affect Prophet's predictions?

- **Answer** : Yes, it absolutely affects the predictions. Here’s how:

    - **Linear Trend**: Assumes that growth continues indefinitely at a constant rate.
        - Future predictions will show a strong upward trend, even if the data is slowing down.
        - Example: If you fit a linear trend to deposit levels, Prophet will extrapolate a steadily increasing deposit level far into the future.

    - **Logistic Trend**: Assumes growth slows as it approaches a saturation point. 
        - Future predictions will show a more tamed upward trend, leveling off over time.
        - Example: If deposits are approaching a plateau, a logistic trend will reflect that in future predictions, ensuring the model doesn't over-predict growth.

- **Key Insight**:If you fit Prophet with the wrong trend type (e.g., linear when the data shows signs of saturation), your forecasts will misrepresent the future. Context is critical here: If you know from domain expertise or external analysis that growth will saturate, using a logistic trend will give more realistic predictions.

#### a. Formatting the time as a date instead of a character and also have anumeric format of the date

In [ ]:
# Step 1: Format time as date 
# Format
time <- as.Date(as.yearqtr(deposit_levels$time, format = "%YQ%q"))
# Add the formatted date back into the dataset
deposit_levels$time_as_date <- time

# Step 2: Format time as numeric
# Format
time2 <- as.numeric(deposit_levels$time_as_date)
#  Add the formatted date back into the dataset
deposit_levels$time_as_numeric <- time2

In [ ]:
saveRDS(deposit_levels, "deposits.rds")  # Save the data to call it in other notebooks

In [ ]:
# Step 3: Plot the model
ggplot(deposit_levels, aes(x = time_as_date, y = deposits)) + 
  geom_point(color = "blue") +
  labs(title = "Deposit Levels from 1980 to 2024 (in $)", 
       x = "Time", 
       y = "Deposit Level (in $)") +
  theme_classic() + 
  theme(
    plot.title = element_text(size = 16, face = "bold"), # Increase title size
    axis.title.x = element_text(size = 12),             # Increase x-axis label size
    axis.title.y = element_text(size = 12),             # Increase y-axis label size
    axis.text = element_text(size = 12)                 # Increase axis tick labels size
  )

#### b. Visualizing Linear Trend

In [ ]:
# Step 1: Time needs to be formatted as numeric for it to work ==> DONE above

# Step 2: Fit the linear model
linear_model <- lm(deposits ~ time_as_numeric ,data = deposit_levels) # Use time as numeric here

# Step 3: Plot the model
ggplot(deposit_levels, aes(x = time_as_date, y = deposits)) + 
  geom_point() +
  geom_smooth(method = lm, col = "green", linewidth = 3) + 
  labs(title = "Linear Fit of Deposit Levels (in $)", 
       x = "Time", 
       y = "Deposit Level (in $)") +
  theme_classic() + 
  theme(
    plot.title = element_text(size = 20, face = "bold"), # Increase title size
    axis.title.x = element_text(size = 14),             # Increase x-axis label size
    axis.title.y = element_text(size = 14),             # Increase y-axis label size
    axis.text = element_text(size = 12)                 # Increase axis tick labels size
  )

#### c. Visualizing Logistic Growth Trend

- Not the same as logistic resgression

- A mathematical model used to describe non-linear growth that slows as it approaches a saturation point.

In [ ]:
# Step 1: Define a logistic function
logistic_function <- function(x, L, k, x0) {
  L / (1 + exp(-k * (x - x0)))  # L = carrying capacity, k = growth rate, x0 = midpoint
}

# Step 2: Fit the logistic model
logistic_model <- nls(deposits ~ logistic_function(time_as_numeric, L, k, x0),
                      data = deposit_levels,
                      start = list(L = max(deposit_levels$deposits), k = 0.01, x0 = median(deposit_levels$time_as_numeric)))


# Step 3: Add logistic trend predictions to the dataset
deposit_levels <- deposit_levels %>%
  mutate(logistic_trend = predict(logistic_model))

# Step 4: Plot the linear model
ggplot(deposit_levels, aes(x = time_as_date, y = deposits)) +
  geom_point(color = "black", linewidth = 1) +  # Actual data
  geom_line(aes(y = logistic_trend), color = "red", linewidth = 3) +  # Logistic trend
  labs(title = "Logistic Trend Fit of Deposit Levels (in $)", 
       x = "Date", 
       y = "Deposits (in $)") +
  theme_classic() +  # Simplify the appearance
  theme(
    plot.title = element_text(size = 18, face = "bold"), # Increase title size
    axis.title.x = element_text(size = 14),             # Increase x-axis label size
    axis.title.y = element_text(size = 14),             # Increase y-axis label size
    axis.text = element_text(size = 12)                 # Increase tick label size
  )

#### d. Conclusion for Trend

**Why NOT logistic trend model?**

- Even though, the logistic trend line seems to fit better past data, the logistic trend assumes market saturation and plateau. 
- But due to contextual information we have, we know that this is not a plateau but the slow growth was due to the 2008 crisis which is a one time event and not a recurring structural pattern.
- If we pick logistic trend, it might lead to under-predictions if the future involves a period of growth.
- This highlights the importance of blending data patterns with domain insights. A blind reliance on "best fit" could lead to a misinterpretation of the trends.

**Why pick linear trend fit?**
- Linear trends assume consistent growth over time, which aligns well with long-term economic expansion and deposit accumulation in a stable financial system
- Plus, current high interest rate environment incentivizes deposits.
- We see a continued economic recovery post 2008 crisis, suggesting deposits will grow, though slower than pre-crisis period.
- Uncertainty about inflation and the economy may drive precautionary savings since we are also recovering from the covid 19 crisis.
- Ultimately, choosing linear growth aligns with the economic rationale that deposits will likely grow as part of the recovery phase. 

**Additional Data Manipulation?**
- Will we have to use log? Or will that change the linearity?
- Applying a log transformation to your data can help make it fit better if your deposit levels grow exponentially rather than linearly. However, this doesn't change the fact that you're still modeling a linear trend—it just adjusts the scale to make it easier for Prophet to handle the data
- If your data already behaves linearly, logging may unnecessarily distort it. ==> I think it's fine


In [ ]:
# Code to check for linearit assumption
linear_model1 <- lm(deposits ~ time_as_numeric, data = deposit_levels) # Linearity is struggling due to changepoints but I can add changepoints to prophet
#plot(linear_model1)
# Transform with log and see if it's better
linear_model2 <- lm(log(deposits) ~ time_as_numeric, data = deposit_levels) # No it did not help improve
#plot(linear_model2)

### Step 3: Seasonality Expectation

- Tools like Prophet and others can explicitly model seasonality if detected, which improves forecasting by capturing these periodic variations.

- However it's important to manually double check and understand the context behind the model.

- If seasonality exists but isn’t accounted for in the model, predictions may be inaccurate.

- For instance, if deposits consistently increase in Q4 but your model doesn’t include seasonality, it will underestimate future deposits in that quarter.

#### a. Let's plot a detrended ACF to check for seasonality

**How to read an ACF Plot**

- We look at the peaks that go above the blue line. Check lags based on the frequency:
   - Lag = 12 for monthly data (12 months in a year)
   - Lag = 4 for quarterly data
   - Lag = 52 for weekly data

In [ ]:
# Step 1: Convert deposits into a time series object. ACF only takes in time series data. 
deposit_time_series <- ts(deposit_levels$deposits, 
                        start = c(1984, 1), # we could have put deposit_levels$time_as_numeric but later on when plotting randomness the x axis won't show time properly so we just hardcode to tell where X axis will start and with frequency it knows how to link X and Y. 
                        frequency = 4) # Quarterly frequency 

# Step 2: Subtract the trend component, since the trend dominates, remove it to isolate the seasonality
        # Decompose the trend: breaks down the trend
deposit_decompose_trend <- decompose(deposit_time_series, type="additive") # breaks down dataset into 4 categories: observed data, trend, seasonality and randomness plot(deposit_decompose_trend) to see all 4 
        # Remove the trend from the time series data
deposit_detrended <- deposit_time_series - deposit_decompose_trend$trend

# Step 3: Plot ACF to check for seasonality
acf(deposit_detrended, na.action = na.omit, main = "Autocorrelation Plot of the Detrended of Deposits") # NAs comes from decomposition algorithm uses a moving window to calculate the trend, and it cannot compute values for the edges of the series.

#### b. Conclusion for Seasonality

- There are no peaks above the blue line at lag 4 (Quarterly Seasonality), the absence of such spikes suggests that quarterly seasonality is weak or not present in this case.

- For now, we will let Prophet's built-in seasonal detection handle any weak seasonal patterns

### Step 4: Outliers Expectation

**Why check for outliers**:  

- Prophet is designed to handle outliers and is fairly robust to their presence.

- However, extreme anomalies (like sudden, sharp increases or drops) can still affect the trend and seasonal components, distorting predictions.

In [ ]:
# Check steps here: https://sqlpad.io/tutorial/interquartile-range/#:~:text=The%20standard%20method%20involves%20calculating,these%20bounds%20are%20potential%20outliers.

# Step 1: Calculate the IQR
IQR_value <- IQR(deposit_levels$deposits)

# Step 2: Define Boundaries
lower_bound <- quantile(deposit_levels$deposits, 0.25) - 1.5 * IQR_value
upper_bound <- quantile(deposit_levels$deposits, 0.75) + 1.5 * IQR_value

# Step 3: Flag Outliers
outliers <- subset(deposit_levels, deposits < lower_bound | deposits > upper_bound)

# step 4: Print Outliers
outliers

**Conclusion for Outliers:**

- Based of the results, it looks like the IQR method did not flag any outliers in the data. The deposit levels in the dataset fall within the acceptable range defined by the IQR method. The deposit levels may be naturally smooth without extreme spikes or dips (maybe lol?)

- Even without detected outliers, we should remain aware of contextual anomalies (e.g., the 2008 crisis) that could still affect model interpretation and trends.

### Step 5: Noise


**Why check for noise?**

- Noise refers to the unexplained or irrelevant variation in the data that cannot be attributed to meaningful patterns.
- Prophet is robust to noise, but excessive noise may require preprocessing (e.g., smoothing).
- A higher proportion (close to 1) indicates the data is dominated by noise.
- A lower proportion (closer to 0) means the data has more structured patterns (e.g., trend, seasonality).

In [ ]:
# Calculate the variance of the residuals to quantify the noise and Compare this variance to the overall variance in the observed data.

# Variance of residuals (randomness)
residual_variance <- var(deposit_decompose_trend$random, na.rm = TRUE) # Use our decomposed dataset

# Variance of the original data
observed_variance <- var(deposit_time_series, na.rm = TRUE)  # Use our dataset that was transformed to time series, if we use deposit_levels directly it won't work

# Proportion of variance due to noise
global_noise_proportion <- residual_variance / observed_variance
global_noise_proportion

**Conclusion for Noise**

- Our noise proportion of approximately 0.00067 (a very small number), which means: noise contributes very little to the overall variability in your data. 
- This is typical seen when our data has a strong long-term trend.
- So there won't be any need for excessive smoothing or denoising steps. Prophet can effectively handle the low-level noise.

#### Step 6: Localised Randomness

**Why check for randomness?**

-  Randomness, as shown in decomposition, includes localized irregularities or residual variations that aren't explained by the trend or seasonality but may include event-driven or one-off effects (like the 2008 crisis). It’s more localized and descriptive of specific periods rather than a global property like noise.
- While Prophet is robust to some noise, high randomness (localized irregularities) can make it harder to discern meaningful patterns.

In [ ]:
# plot(deposit_decompose_trend) ==> from the ACF code above in step 3.a

plot(deposit_decompose_trend$random,
    xlab = "Time",
    ylab = "Random Component",
    main = "Random Component of Deposits over Time")

**Conclusion for Randomness**
- There is significant randomness during the 2008 crisis period.
- We can add on a changepoint to handle abrupt trend changes.
- Prophet will automatically detects changepoints in the trend. However, you can manually specify changepoints (e.g., around 2008) where the data exhibits high variability.

## Part 3: Forecasting with Prophet

In [ ]:
prophet_data_1 <- deposit_levels[, c("deposits", "time_as_date")]

### Step 2: Prepared the Data

In [ ]:
# Step 1: Prepare data for prophet to work

# you have to rename them as ds and y for prophet to work and it will automatically ignore all other columns ==> RULE
prophet_data_1$ds <- as.Date(prophet_data_1$time_as_date)
prophet_data_1$y <- prophet_data_1$deposits

### Step 3: Fit Prophet on training data

In [ ]:
# Step 1: Initialize the model
prophet_model<- prophet(changepoints = c("2008-01-01")) # Add Customizations based on assumptions made above https://stackoverflow.com/questions/64822488/how-to-use-prophets-make-future-dataframe-with-multiple-regressors
# We can later add on additional regressor to the best model

# Step 2: Fit the model: Training the model
prophet_model <- fit.prophet(prophet_model, prophet_data_1) # syntax

### Step 4: Create a Future Dataframe

In [ ]:
future_data <- make_future_dataframe(prophet_model, periods = 33, freq = "quarter") # start counting from the end of the training data: 2020 Q4 to 2028 Q1
#future_data # The output of make_future_dataframe() will include both the training data dates and the newly created future dates.

### Step 5: Predict Future Values

In [ ]:
prediction_data <- predict(prophet_model, future_data)

### Step 6: Visualize the Forecast

In [ ]:
# Ensure 'ds' is a Date object
prophet_data_1$ds <- as.Date(prophet_data_1$ds)
prediction_data$ds <- as.Date(prediction_data$ds)

ggplot() +
  # Add historical data points
  geom_point(data = prophet_data_1, aes(x = ds, y = y, color = "Historical Data"), size = 1.5, alpha = 0.8) +
  # Add the predicted trend line
  geom_line(data = prediction_data, aes(x = ds, y = yhat, color = "Predicted Trend"), size = 2) +
  # Add uncertainty intervals for predictions
  geom_ribbon(
    data = prediction_data,
    aes(x = ds, ymin = yhat_lower, ymax = yhat_upper, fill = "Uncertainty Interval"),
    alpha = 0.2
  ) +
  # Add labels and a title
  labs(
    title = "Historical Data and Prediction with Prophet",
    x = "Date",
    y = "Deposits (in $)",
    color = "Legend",
    fill = "Legend"
  ) +
  # Customize colors for the legend
  scale_color_manual(
    values = c("Historical Data" = "black", "Predicted Trend" = "red")
  ) +
  scale_fill_manual(
    values = c("Uncertainty Interval" = "blue")
  ) +
  # Add a minimal theme
  theme_minimal() +
  theme_classic() +
  theme(
    plot.title = element_text(hjust = 0.5, size = 14, face = "bold"),
    axis.title.x = element_text(size = 12),
    axis.title.y = element_text(size = 12),
    legend.title = element_text(size = 12, face = "bold"),
    legend.text = element_text(size = 10)
  )

In [ ]:
prophet_plot_components(prophet_model, prediction_data)

## Part 4: Evaluation of the Model Performance

In [ ]:

# Step 1: Split Data between training and test data
train_data <- subset(prophet_data_1, ds <= "2019-12-31") # Training data: 1984 Q1 to 2020 Q4
test_data <- subset(prophet_data_1, ds > "2019-12-31")  # Testing data: 2021 Q1 to 2024 Q2

# Step 2: Prepare data for prophet to work
# you have to rename them as ds and y for prophet to work and it will automatically ignore all other columns ==> RULE
train_data$ds <- as.Date(train_data$ds)
train_data$y <- train_data$y

test_data$ds <- as.Date(test_data$ds)
test_data$y <- test_data$y

### Step 1: Compare predictions with test data 

- The test data is unseen data (not used in training) that allows you to assess how well your model generalizes to new data.
- Comparing predictions to actual values in the test data helps identify whether the model overfits or underfits.
- A good forecast should perform well not only on the training data but also on the test data. This is a crucial check before relying on the model for real-world forecasting.

In [ ]:
# Step 1: Create future dataframe for test data
future_test_data <- make_future_dataframe(prophet_model, periods = nrow(test_data), freq = "quarter") # Testing data: 2021 Q1 to 2024 Q2
#future_test_data

# Step 2: Make predictions for the test data
prediction_test_data <- predict(prophet_model, future_test_data) 
prediction_test_data$ds <- as.Date(prediction_test_data$ds) #make sure ds is in date format or else merge will fail

# Step 3: Merge predictions with actual test data: 
# The merging step combines your actual test data with the predictions generated by Prophet. This creates a single dataframe where you can directly compare the actual (y) and predicted (yhat) values for each time point (ds)
prediction_test_results <- merge(test_data, prediction_test_data[, c("ds", "yhat")], by = "ds")

# Step 4: Plot the predictions vs actuals
ggplot(prediction_test_results, aes(x = ds)) +
  geom_line(aes(y = y, color = "Actual"), size = 1) + # Actual test data
  geom_line(aes(y = yhat, color = "Predicted"), size = 1, linetype = "dashed") + # Predictions
  labs(
    title = "Predictions vs Actuals on Test Data with Prophet Model",
    x = "Time",
    y = "Deposit Levels",
    color = "Legend"
  ) +
  theme_classic()

**Observations**
- The predicted model (blue dashed line) done with Prophet does not closely align with the test (actual) data from 2021 to 2024. 
- There's a significant divergence between the actuals and predictions: the model overpredicts significantly in this range, showing an upward trend, whereas the actual values flatten or slightly decrease.

### Step 2. Compute Perfomance Metrics

In [ ]:
# Mean Absolute Error (MAE)

actual <- prediction_test_results$y
predicted <- prediction_test_results$yhat

MAE_prophet <- mean(abs(actual - predicted))
MAE_prophet

# Root Mean Squared Error (RMSE)
RMSE_prophet <- sqrt(mean((actual - predicted)^2))
RMSE_prophet

# Mean Absolute Percentage Error (MAPE)
MAPE_prophet <- mean(abs(actual - predicted) / actual) * 100
MAPE_prophet

# Compute lagged actual values
lagged_actual <- c(NA, head(actual, -1))

# Calculate MAE of naive forecast (excluding first NA value)
MAE_naive <- mean(abs(actual[-1] - lagged_actual[-1]), na.rm = TRUE)

# Calculate MASE
MASE_prophet <- MAE_prophet / MAE_naive
MASE_prophet

# Part 5: Adding on External Regressors to the Model

## Step 1: Select Relevant Features through domain knowledge

- Our goal here is research features that are likely to affect deposit level
- To narrow our options we will use a mix of domain knowledge and a LASSO regression to undertand the relationship between deposit and those potential regressors

> Here is a list of features through reserach and domain knowledge that I believe most impact deposit levels:
- GDP Growth Rate: Stronger economic growth can lead to higher income and savings
- Unemployment Rate: Higher unemployment can reduce deposits due to lower income
- Consumer Confidence Index: Reflects consumers' willingness to save vs. spend
- Population Growth: Larger populations in a region can increase deposit levels over time
- 10 Year Treasury Yield: Represent the interest rate environment, directly affecting deposit rates and competing as an alternative safe investment option.
- Federal Funds Effective Rate: impacts savings account interest rate
- Disposable Personal Income (percent change from year ago): ok
- Consumer Price Index (units: growth rate previos period in decimal): measure of inflation rate. Higher inflation may reduce disposable income and savings levels

## Step 2: Narrow Down Features through LASSO Regression

### A. Prepare the Data

- Combine deposit level data and regressors data into one => deposit_regressor_data
- Keeping the target variable (Deposit_Levels) untouched.
- Ensuring non-regressor columns (e.g., time) are excluded from scaling. time: A timestamp or date column (used for splitting but not for scaling or modeling).
- Scaling only the features (regressors). Other columns (GDP_Growth, Unemployment_Rate, Interest_Rate, SP500, etc.): Features to be scaled.

In [ ]:
# Combine deposit level data and regressors data into one => deposit_regressor_data
deposit_regressor_data <- merge(deposit_levels, regressors_data, by = c("time_as_date", "time_as_numeric"))

# Pre process only the regressors inside your data

# Exclude all that's not a regressor like the target varaiable (deposit) and time variables
regressors_col <- setdiff(names(deposit_regressor_data), c("logistic_trend", "time", "time_as_date", "time_as_numeric", "deposits"))

# The preProcess function centers (mean = 0) and scales (std = 1) your predictors => Apply scaling to the feature columns
pre_process_data <- preProcess(deposit_regressor_data[, regressors_col], method=c("center", "scale"))

## B. Split dataset into Training dataset and Test dataset

In [ ]:
# Split data into training and testing based on time
train_data_regressor <- subset(deposit_regressor_data, time_as_date <= "2019-12-31") # Training data: 1984 Q1 to 2020 Q4
test_data_regressor <- subset(deposit_regressor_data, time_as_date > "2019-12-31")  # Testing data: 2021 Q1 to 2024 Q2

# Apply the pre-processing to our training and testing dataset

# Scale training data (apply preprocessing to the regressor columns only)
train_data_scaled <- train_data_regressor
train_data_scaled[, regressors_col] <- predict(pre_process_data, train_data_regressor[, regressors_col])

# Scale testing data using the same preprocessing parameters
test_data_scaled <- test_data_regressor
test_data_scaled[, regressors_col] <- predict(pre_process_data, test_data_regressor[, regressors_col])

### C. Fit the LASSO Model

In [ ]:
# Create matrices for modeling for training data

X_train_lasso <- as.matrix(train_data_scaled[, regressors_col])
X_train_lasso[is.na(X_train_lasso)] <- 0 # fill the mssing value with 0
y_train_lasso <- train_data_scaled$deposits

# Create matrices for modeling for testing data
X_test_lasso <- as.matrix(test_data_scaled[, regressors_col])
y_test_lasso <- test_data_scaled$deposits

# Fit the lasso_model
lasso_model <- cv.glmnet(X_train_lasso, y_train_lasso, alpha = 1)

# Extract selected features at lambda.min
selected_features <- coef(lasso_model, s = "lambda.min")
print(selected_features)

- You’ve chosen not to scale the deposits column, which is fine as long as you don’t intend to interpret the coefficients from the LASSO model directly. However, if coefficient interpretation is needed, it’s often better to scale the target variable as well (standardize deposits).
-  Lasso revealed that only unemployment rate and GDP are useful here.

# Part 6: Add the regression into our model

## Step 1: Plugging Regressors into Prophet

- With a MAPE of 10%, it’s not ideal, but it’s not a dealbreaker
- Even if the Lasso model's MAPE is high, the regressors may still contain useful signals that Prophet can leverage.
- Prophet doesn't rely solely on the regressors—it also models time-based trends and seasonality, which can compensate for some of the inaccuracy in the regressors.

In [ ]:
#Step 1: create dataframe

prophet_data_2 <- data.frame(
    ds = deposit_regressor_data$time_as_date,
    y = deposit_regressor_data$deposits,
    GDP = deposit_regressor_data$GDP,
    unemployment_rate = deposit_regressor_data$unemployment_rate,
    fed_rate = deposit_regressor_data$fed_rate,
    ConsumerPriceIndex = deposit_regressor_data$ConsumerPriceIndex # stop here if it makes things worse
    )

# Step 2: Handle Consumer price index last missing value
prophet_data_2$ConsumerPriceIndex[is.na(prophet_data_2$ConsumerPriceIndex)] <- mean(prophet_data_2$ConsumerPriceIndex, na.rm = TRUE)

# Step 3: Initialize the model by adding on regressors and Fit Prophet

# Initialize the model
prophet_model_2 <- prophet()

# Add regressors
prophet_model_2 <- add_regressor(prophet_model_2, 'GDP')
prophet_model_2 <- add_regressor(prophet_model_2, 'unemployment_rate')
prophet_model_2 <- add_regressor(prophet_model_2, 'ConsumerPriceIndex')
prophet_model_2 <- add_regressor(prophet_model_2, 'fed_rate')

# Step 4: Fit the model: full data the model
prophet_model_2 <- fit.prophet(prophet_model_2, prophet_data_2) # syntax

# Step 5: Create a future dataframe
future_data_2 <- make_future_dataframe(prophet_model_2, periods = 15, freq = "quarter") # start counting from the end of the training data: 2020 Q4 to 2028 Q1
#future_data_2  # The output of make_future_dataframe() will include both the training data dates and the newly created future dates.future_train_data_2

In [ ]:
# Step 1: Create dataframes for GDP and unemployment rate
gdp_data <- data.frame(
  ds = prophet_data_2$ds, # Use the same date column as in your training data
  y = prophet_data_2$GDP  # Use the GDP column
)

unemployment_data <- data.frame(
  ds = prophet_data_2$ds, # Use the same date column as in your training data
  y = prophet_data_2$unemployment_rate # Use the unemployment rate column
)

fed_data <- data.frame(
  ds = prophet_data_2$ds, # Use the same date column as in your training data
  y = prophet_data_2$fed_rate # Use the fed rate column
)

CPI_data <- data.frame(
  ds = prophet_data_2$ds, # Use the same date column as in your training data
  y = prophet_data_2$ConsumerPriceIndex # Use the CPI column
) 

# Step 2: Fit Prophet models for GDP and unemployment rate
gdp_model <- prophet(gdp_data) # Fit the GDP model
unemployment_model <- prophet(unemployment_data) # Fit the unemployment rate model
fed_model <- prophet(fed_data)
CPI_model <- prophet(CPI_data)

# Step 3: Generate future dataframes matching the number of rows in future_data_2
future_gdp <- make_future_dataframe(gdp_model, periods = nrow(future_data_2) - nrow(prophet_data_2), freq = "quarter")
future_unemployment <- make_future_dataframe(unemployment_model, periods = nrow(future_data_2) - nrow(prophet_data_2), freq = "quarter")
future_fed <- make_future_dataframe(fed_model, periods = nrow(future_data_2) - nrow(prophet_data_2), freq = "quarter")
future_CPI <- make_future_dataframe(CPI_model, periods = nrow(future_data_2) - nrow(prophet_data_2), freq = "quarter")

# Step 4: Forecast the future regressor values
gdp_forecast <- predict(gdp_model, future_gdp)
unemployment_forecast <- predict(unemployment_model, future_unemployment)
fed_forecast <- predict(fed_model, future_fed)
CPI_forcast <- predict(CPI_model, future_CPI)

# Step 5: Extract forecasted values for GDP and unemployment rate
future_GDP_values <- gdp_forecast$yhat[(nrow(prophet_data_2) + 1):nrow(future_gdp)]
future_unemployment_values <- unemployment_forecast$yhat[(nrow(prophet_data_2) + 1):nrow(future_unemployment)]
future_fed_rate_values <- fed_forecast$yhat[(nrow(prophet_data_2) + 1):nrow(future_fed)]
future_CPI_values <- CPI_forcast$yhat[(nrow(prophet_data_2) + 1):nrow(future_CPI)]

# Step 6: Add future GDP and unemployment rate values to future_data_2
future_data_2$GDP <- c(prophet_data_2$GDP, future_GDP_values) # Combine training and future GDP values
future_data_2$unemployment_rate <- c(prophet_data_2$unemployment_rate, future_unemployment_values) # Combine training and future unemployment rate values
future_data_2$fed_rate <- c(prophet_data_2$fed_rate, future_fed_rate_values) 
future_data_2$ConsumerPriceIndex <- c(prophet_data_2$ConsumerPriceIndex, future_CPI_values) 

# Step 7: Verify the updated future_data_2

head(future_data_2) # Check the final dataframe with added co
tail(future_data_2)

In [ ]:
# Step 8: Predict Future Values
prediction_data_2 <- predict(prophet_model_2, future_data_2)

In [ ]:
saveRDS(prediction_data_2, "prediction_data_2.rds")  # Save the data to call it in other notebooks

In [ ]:
prophet_plot_components(prophet_model_2, prediction_data_2)

## Step 2: Visualize the foreacast

In [ ]:
# Ensure 'ds' is a Date object
prophet_data_2$ds <- as.Date(prophet_data_2$ds)
prediction_data_2$ds <- as.Date(prediction_data_2$ds)

ggplot() +
  # Add historical data points
  geom_point(data = prophet_data_2, aes(x = ds, y = y, color = "Historical Data"), size = 1.5, alpha = 0.8) +
  # Add the predicted trend line
  geom_line(data = prediction_data_2, aes(x = ds, y = yhat, color = "Predicted Trend"), size = 2) +
  # Add uncertainty intervals for predictions
  geom_ribbon(
    data = prediction_data_2,
    aes(x = ds, ymin = yhat_lower, ymax = yhat_upper, fill = "Uncertainty Interval"),
    alpha = 0.2
  ) +
  # Add labels and a title
  labs(
    title = "Historical Data and Prediction with Prophet",
    x = "Date",
    y = "Deposits (in $)",
    color = "Legend",
    fill = "Legend"
  ) +
  # Customize colors for the legend
  scale_color_manual(
    values = c("Historical Data" = "black", "Predicted Trend" = "red")
  ) +
  scale_fill_manual(
    values = c("Uncertainty Interval" = "blue")
  ) +
  # Add a minimal theme
  theme_classic() +
  theme(
    plot.title = element_text(hjust = 0.5, size = 14, face = "bold"),
    axis.title.x = element_text(size = 12),
    axis.title.y = element_text(size = 12),
    legend.title = element_text(size = 12, face = "bold"),
    legend.text = element_text(size = 9)
  )

## Step 3: Evaluation of the Model Performance

In [ ]:

# Step 1: Split Data between training and test data
train_data_2 <- subset(prophet_data_2, ds <= "2019-12-31") # Training data: 1984 Q1 to 2020 Q4
test_data_2 <- subset(prophet_data_2, ds > "2019-12-31")  # Testing data: 2021 Q1 to 2024 Q2

# Step 2: Prepare data for prophet to work
# you have to rename them as ds and y for prophet to work and it will automatically ignore all other columns ==> RULE
train_data_2$ds <- as.Date(train_data_2$ds)
train_data_2$y <- train_data_2$y

test_data_2$ds <- as.Date(test_data_2$ds)
test_data_2$y <- test_data_2$y

In [ ]:
# Step 3: Initialize the model by adding on regressors and Fit Prophet

# Initialize the model
prophet_model_test <- prophet()

# Add regressors
prophet_model_test <- add_regressor(prophet_model_test, 'GDP')
prophet_model_test <- add_regressor(prophet_model_test, 'unemployment_rate')
prophet_model_test <- add_regressor(prophet_model_test, 'ConsumerPriceIndex')
prophet_model_test <- add_regressor(prophet_model_test, 'fed_rate')

# Step 4: Fit the model: Training the model
prophet_model_test <- fit.prophet(prophet_model_test, train_data_2) # syntax

# Step 5: 
future_test_data_2 <- make_future_dataframe(prophet_model_test, periods = nrow(test_data_2), freq = 'quarter')

# Step 6: Train models for GDP and unemployment rate, CPI and fed rate
# You do not need to forcast them since here you alraedy have the values for the test dataframe
future_test_data_2$GDP <- test_data_2$GDP
future_test_data_2$unemployment_rate <- test_data_2$unemployment_rate
future_test_data_2$ConsumerPriceIndex <- test_data_2$ConsumerPriceIndex
future_test_data_2$fed_rate <- test_data_2$fed_rate

# Step 7: Predict for the test period
forecast_test <- predict(prophet_model_test, future_test_data_2)

In [ ]:

# Merge predictions with actual test data: 

forecast_test$ds <- as.Date(forecast_test$ds) #make sure ds is in date format or else merge will fail
test_data_2$ds <- as.Date(test_data_2$ds)

# The merging step combines your actual test data with the predictions generated by Prophet. This creates a single dataframe where you can directly compare the actual (y) and predicted (yhat) values for each time point (ds)
forecast_test_results <- merge(test_data_2, forecast_test[, c("ds", "yhat")], by = "ds")
tail(forecast_test_results)

In [ ]:
# Calculate naive forecast
naive_forecast <- c(NA, head(test_data_2$y, -1))  # Shift y_test by one step for naive forecast

# Create a data frame for plotting
naive_vs_model <- data.frame(
  Date = test_data_2$ds,  # Dates from test data
  Actual_Deposits = test_data_2$y,  # Actual deposit values
  Naive_Forecast = naive_forecast,  # Naive forecast values
  Predicted_Deposits = forecast_test_results$yhat  # Predicted deposit values
)

# Plot the data
library(ggplot2)
ggplot(naive_vs_model, aes(x = Date)) +
  geom_line(aes(y = Actual_Deposits, color = "Actual Deposits"), size = 2) +  # Line for actual deposits
  geom_line(aes(y = Naive_Forecast, color = "Naive Forecast"), size = 2, linetype = "dotted") +  # Dotted line for naive forecast
  geom_line(aes(y = Predicted_Deposits, color = "Predicted Deposits"), size = 2, linetype = "dashed") +  # Dashed line for model predictions
  labs(
    title = "Actual vs Naive vs Predicted Deposits (2020-2024)",
    x = "Date",
    y = "Deposits (in $)",
    color = "Legend"
  ) +
  theme_minimal() +
  theme(
    plot.title = element_text(hjust = 0.5, size = 14, face = "bold"),
    axis.title = element_text(size = 12),
    legend.title = element_text(size = 12, face = "bold"),
    legend.text = element_text(size = 10)
  )